In [5]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import numpy as np

# Load model
model_id = "patrickjohncyh/fashion-clip"
model = CLIPModel.from_pretrained(model_id)
processor = CLIPProcessor.from_pretrained(model_id)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") # mac lol
model = model.to(device)

config.json:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [6]:
import csv
import ast

with open('../data/streetwear.csv', newline='', encoding='utf-8') as csvfile:
  reader = csv.DictReader(csvfile)
  data = [row for row in reader]

for row in data:
  row["Comments"] = ast.literal_eval(row["Comments"]) # maybe we find another way of storing the comments
data

[{'Title': 'Finished radiation for my brain tumor today🙌🏻💪🏻',
  'Description': '💪🏻💪🏻',
  'Score': '104494',
  'URL': 'https://i.redd.it/y0l77hifhufe1.jpeg',
  'Author': 'itsthreem',
  'Created UTC': '2025-01-29 02:34:40',
  'Images': 'https://i.redd.it/y0l77hifhufe1.jpeg',
  'Comments': [{'author': 'itsthreem',
    'body': 'Brain hat- etsy\nCrewneck- artisan apparel\nBelt-half evil\nJeans- levi silver tabs\nShoes-marc nolan x kodone collab\nNecklace-inferno studios',
    'score': 885,
    'created_utc': '2025-01-29 02:36:04'},
   {'author': 'zacsmashyou',
    'body': 'Sweater is DOPE',
    'score': 180,
    'created_utc': '2025-01-29 02:38:24'},
   {'author': 'coyote-fever-dream',
    'body': 'Lookin fly! Congrats!!!!',
    'score': 138,
    'created_utc': '2025-01-29 02:43:31'},
   {'author': 'ferretbeast',
    'body': 'Everything about this is awesome! Your look, your smile, you finishing radiation. Amazing! Congrats!!',
    'score': 110,
    'created_utc': '2025-01-29 04:19:05'},
  

In [5]:
#getting comments sentiment analysis score

#1. Importing model
#used a huggingface sentiment analysis multilingual transformer. 
#Selected this one because of fine-grained classification of sentiments
#is multilingual distilbert fine-tuned into a sentiment analysis model
from transformers import pipeline
# Load the classification pipeline with the specified model
pipe = pipeline(model="tabularisai/multilingual-sentiment-analysis")
#into 512 tokens max
#Number of Classes: 5 (Very Negative, Negative, Neutral, Positive, Very Positive)
#Score: model's confidence in its result



config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


<class 'dict'>


In [13]:
#2 getting sentiment scores from individual comments in row[Comments]
# preparing output
sentiment_list = [] #list of dict:{average label: real from 1-5, average score: }
sentiment_mapping = {
    1: "Very Negative",
    2: "Negative",
    3: "Neutral",
    4: "Positive",
    5: "Very Positive"
}
numerical_mapping = {v: k for k, v in sentiment_mapping.items()}
progress = 0
#processing
for row in data:
    total_weightage = 0
    total_result = 0
    comments_list = []
    score_list = []
    for comment in row['Comments']:
        truncated_comment = comment['body'][:1500]
        comments_list.append(truncated_comment)
        score_list.append(comment['score'])
    result = pipe(comments_list)
    for i in range(len(comments_list)):
        label = numerical_mapping[result[i]['label']]
        weighted_result = score_list[i]*label
        total_weightage += score_list[i]
        total_result += weighted_result
    post_score = total_result/total_weightage
    sentiment_list.append(post_score)
    print(progress)
    progress = progress+1

#list of weighted comment sentiment in sentiment_list
#print(sentiment_list)
print(len(sentiment_list))
    
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
#embedding
def run_embedding(images, titles, comments=None):
    """
    Takes in a list of PIL images and a list of texts (length <77 tokens each)
    Returns the image and text embedding vectors for the batch
    """
    inputs = processor(
        text=titles,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=77
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    image_embeds = outputs.image_embeds
    text_embeds = outputs.text_embeds

    image_embeds = torch.nn.functional.normalize(image_embeds)
    text_embeds = torch.nn.functional.normalize(text_embeds)

    return image_embeds, text_embeds


def combine_embeddings(image_embeds, text_embeds):
    return 0.5 * image_embeds + 0.5 * text_embeds # weighted average

In [8]:
# test mongo connection
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
from dotenv import load_dotenv
load_dotenv()

uri = os.environ.get("MONGODB_URI")

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
  client.admin.command('ping')
  print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
  print(e)

client.close()

Pinged your deployment. You successfully connected to MongoDB!


In [13]:
import requests
from PIL import Image
from io import BytesIO

from tqdm import tqdm
import gc

# embedded = []

client = MongoClient(uri, server_api=ServerApi('1'))
db = client["fit-check"]
collection = db["embeddings"]

batch_size = 16 

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

for batch_rows in tqdm(list(batch(data, batch_size))):
    images = []
    titles = []
    comments_list = []
    image_urls = []
    scores = []
    for row in batch_rows:
        image_url = row["Images"].split(",")[0]
        if image_url == "No images":
            continue
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        images.append(image)
        titles.append(row["Title"])
        # You can also batch comments if you want, or just use titles
        comments_list.append([row["Description"]] + [comment["body"] for comment in row["Comments"]])
        image_urls.append(image_url)
        scores.append(row["Score"])

    # Run batch embedding (titles only for simplicity)
    image_embeds, text_embeds = run_embedding(images, titles, None)  # Modify run_embedding to accept lists

    image_embeds_cpu = image_embeds.cpu()
    text_embeds_cpu = text_embeds.cpu()
    
    for idx in range(len(images)):
        collection.insert_one({
            "title": titles[idx],
            "comments": comments_list[idx],
            "image": image_urls[idx],
            "score": scores[idx],
            "image_embeds": {
                "type": "denseVector",
                "values": image_embeds_cpu[idx].tolist(),
            },
            "text_embeds": {
                "type": "denseVector",
                "values": text_embeds_cpu[idx].tolist(),
            },
            "embeds": {
                "type": "denseVector",
                "values": combine_embeddings(image_embeds_cpu[idx], text_embeds_cpu[idx]).tolist(),
            }
        })
        images[idx].close()
    del image_embeds, text_embeds
    gc.collect()

 19%|█▉        | 6/32 [01:46<08:35, 19.81s/it]/Users/ashley/.pyenv/versions/3.10.9/lib/python3.10/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 32/32 [10:06<00:00, 18.95s/it]


thank you gemini for the batching code

In [ ]:
# delete dupes

# Step 1: Aggregate duplicate titles with _ids
pipeline = [
    {
        "$group": {
            "_id": "$title",
            "ids": {"$push": "$_id"},
            "count": {"$sum": 1}
        }
    },
    {
        "$match": {
            "count": {"$gt": 1}
        }
    }
]

duplicates = list(collection.aggregate(pipeline))

total_to_delete = 0
for doc in duplicates:
    title = doc["_id"]
    ids = doc["ids"]
    count = doc["count"]
    ids_to_delete = ids[1:]  # keep one, delete the rest

    print(f"Title: {title}")
    print(f"Occurrences: {count}")
    print(f"Deleting {len(ids_to_delete)} documents with _ids: {ids_to_delete}\n")

    # Uncomment this to delete:
    # collection.delete_many({"_id": {"$in": ids_to_delete}})
    total_to_delete += len(ids_to_delete)

print(f"Total duplicates marked for deletion: {total_to_delete}")


Total duplicates marked for deletion: 0


In [14]:
# Prepare inputs (image + optional text)
compare_text = "blue shirt and brown long pants"
compare_inputs = processor(
    text=[compare_text],
    return_tensors="pt",
    padding=True
)
compare_inputs = {k: v.to(device) for k, v in compare_inputs.items()} 

# Forward pass
with torch.no_grad():
    compare_text_embeds = model.get_text_features(**compare_inputs)

compare_text_embeds = torch.nn.functional.normalize(compare_text_embeds)

In [17]:
similarity = torch.nn.functional.cosine_similarity(image_embeds, text_embeds)
print(f"Image to post.txt similarity: {similarity[0]}")


similarity = torch.nn.functional.cosine_similarity(image_embeds, compare_text_embeds)
print(f"Image to manual description similarity: {similarity[0]}")

similarity = torch.nn.functional.cosine_similarity(text_embeds, compare_text_embeds)
print(f"Manual description to post.txt similarity: {similarity[0]}")

Image to post.txt similarity: 0.2717254161834717
Image to manual description similarity: 0.3059011697769165
Manual description to post.txt similarity: 0.4487333595752716


# previous code (no ai here)

In [ ]:
def run_embedding(image, title, comments):
    """
    Takes in a PIL image and some text (length <77 tokens)
    Returns the image and text embedding vectors separately
    """
    inputs = processor(
        # text=[title, *comments],
        text=[title],
        images=image,
        return_tensors="pt",
        padding=True,
        truncation=True # hm
    )
    inputs = {k: v.to(device) for k, v in inputs.items()} 

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    image_embeds = outputs.image_embeds
    text_embeds = outputs.text_embeds

    image_embeds = torch.nn.functional.normalize(image_embeds)
    text_embeds = torch.nn.functional.normalize(text_embeds)

    return image_embeds, text_embeds

def combine_embeddings(image_embeds, text_embeds):
    return 0.5 * image_embeds + 0.5 * text_embeds # weighted average

In [ ]:
# Run embeddings

import requests
from PIL import Image
from io import BytesIO

from tqdm import tqdm
import gc

embedded = []

# ...existing code...
max_tokens = 77

def truncate_text(text):
    tokens = processor.tokenizer(text, return_tensors="pt")
    if tokens['input_ids'].shape[1] > max_tokens:
        # Truncate to max_tokens
        return processor.tokenizer.decode(tokens['input_ids'][0][:max_tokens])
    return text

i = 0
for row in tqdm(data):
  image_url = row["Images"].split(",")[0] # D:
  if image_url == "No images":
    continue
  response = requests.get(image_url)
  image = Image.open(BytesIO(response.content))

  title = truncate_text(row["Title"])
  comments = [truncate_text(row["Description"])] + [truncate_text(comment["body"]) for comment in row["Comments"]]

  image_embeds, text_embeds = run_embedding(image, title, comments)
  image_embeds_cpu = image_embeds.cpu()
  text_embeds_cpu = text_embeds.cpu()

  del image_embeds, text_embeds
  if i % 20 == 19: gc.collect()
  image.close()

  embedded.append({
    "title": title,
    "comments": comments,
    "image": image_url,
    "score": row["Score"],
    "image_embeds": image_embeds_cpu,
    "text_embeds": text_embeds_cpu,
    "embeds": combine_embeddings(image_embeds_cpu, text_embeds_cpu)
  })
  i += 1

In [ ]:
count = 0
for row in data:
  text = row["Title"]
  tokens = processor.tokenizer(text, return_tensors="pt")
  token_count = tokens['input_ids'].shape[1]  # should be ≤ 77

  if token_count > 77:
    # print(token_count, text)
    count += 1

  for comment in row["Comments"]:
    text = comment["body"]
    tokens = processor.tokenizer(text, return_tensors="pt")
    token_count = tokens['input_ids'].shape[1]  # should be ≤ 77

    if token_count > 77:
      # print(token_count, text)
      count += 1

print(count)
# checking title length
